<div style="text-align: center;">
  <h1 style="background-color: grey; color: white; padding: 10px;"><strong>Price Prediction on Google Cloud Platform </strong></h1>
</div>
<div style="text-align: center;">
<h4>by Shafa Salzabila Meidita </h4></div>

## Library and Data Preparation

In [69]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
import pickle
import pandas as pd

In [70]:
## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/ssalzabilam/shafa-037/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'shafa_dataset_037'
table_id = 'data_to_predict'
region = 'us-central1'
bucket_name = 'shafa_gcs_037'
blob_name = 'data/data_saudi_used_cars.csv'

model_name = 'Model_SaudiArabia_UsedCars.sav'

In [71]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('final_model_saudi_usedcars.sav')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


## Load Data

In [72]:
#load data from BigQuery
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
auto_cloud = query_job.result().to_dataframe()

auto_cloud

/home/ssalzabilam/miniconda3/envs/env-saudi-usedcars/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable
0,A,Riyadh,Mercedes,Manual,Saudi,Full,2020,2.0,5000,True
1,C,Riyadh,Mercedes,Automatic,Other,Full,2017,7.0,40000,True
2,C,Riyadh,Mercedes,Automatic,Saudi,Full,2019,2.0,28000,True
3,C,Makkah,Mercedes,Automatic,Saudi,Full,2016,6.3,50000,True
4,C,Riyadh,Mercedes,Automatic,Saudi,Full,2015,2.0,110000,True
...,...,...,...,...,...,...,...,...,...,...
1791,Land Cruiser Pickup,Riyadh,Toyota,Manual,Saudi,Semi Full,2012,4.0,300000,True
1792,Land Cruiser Pickup,Riyadh,Toyota,Manual,Other,Semi Full,2016,5.0,184,True
1793,Land Cruiser Pickup,Qassim,Toyota,Manual,Gulf Arabic,Semi Full,2012,4.0,300,True
1794,Land Cruiser Pickup,Tabouk,Toyota,Manual,Gulf Arabic,Semi Full,2015,4.0,66000,True


## Predict

In [73]:
# Predict New Data 
new_car = pd.DataFrame({
    'Car Type' : ['Yukon', 'Yukon'], 
    'Manufacturer' : ['GMC', 'GMC'], 
    'Country Manufacturer' : ['USA', 'USA'],
    'Transmission' : ['Automatic', 'Automatic'],
    'Package' : ['Full', 'Full'], 
    'Region' : ['Aseer', 'Aseer'], 
    'Country Origin' : ['Saudi', 'Saudi'], 
    'Year' : [2011, 2021], 
    'Engine Size': [1.0, 6.6], 
    'Mileage' : [256000, 100]
    })

In [74]:
# Load the model
with open('final_model_saudi_usedcars.sav', 'rb') as f:
    loaded_model = pickle.load(f)

predict_new_car = loaded_model.predict(new_car)
predict_new_car

array([ 86587.49, 272288.1 ], dtype=float32)

In [75]:
def preprocessing(df):
    df = df.drop(index=df[df['Origin'] == 'Unknown'].index)
    df = df.reset_index(drop=True)
    
    if 'Negotiable' in df.columns:
        df = df.drop(columns='Negotiable')
        
    df.rename(columns={
        'Type': 'Car Type',
        'Make': 'Manufacturer',
        'Gear_Type': 'Transmission',
        'Origin': 'Country Origin',
        'Options': 'Package',
        'Engine_Size' : 'Engine Size'
    }, inplace = True)
    
    country_manufacturer = {
        'Land Rover': 'United Kingdom',
        'Kia': 'South Korea',
        'Mazda': 'Japan',
        'Porsche': 'Germany',
        'Hyundai': 'South Korea',
        'Toyota': 'Japan',
        'Chrysler': 'USA',
        'Mitsubishi': 'Japan',
        'MG': 'United Kingdom',
        'Chevrolet': 'USA',
        'Mercedes': 'Germany',
        'BMW': 'Germany',
        'Jeep': 'USA',
        'Lincoln': 'USA',
        'Nissan': 'Japan',
        'GMC': 'USA',
        'Ford': 'USA',
        'Genesis': 'South Korea',
        'Zhengzhou': 'China',
        'Dodge': 'USA',
        'HAVAL': 'China',
        'Lexus': 'Japan',
        'Changan': 'China',
        'Aston Martin': 'United Kingdom',
        'Cadillac': 'France',
        'INFINITI': 'Japan',
        'Honda': 'Japan',
        'Audi': 'Germany',
        'BYD': 'China',
        'Volkswagen': 'Germany',
        'Victory Auto': 'China',
        'Suzuki': 'Japan',
        'Renault': 'France',
        'Geely': 'China',
        'Isuzu': 'Japan',
        'Daihatsu': 'Japan',
        'Maserati': 'Italy',
        'GAC': 'China',
        'Chery': 'China',
        'Jaguar': 'United Kingdom',
        'Rolls-Royce': 'United Kingdom',
        'Peugeot': 'France',
        'Foton': 'China',
        'MINI': 'United Kingdom',
        'Fiat': 'Italy',
        'Iveco': 'Italy',
        'Bentley': 'United Kingdom',
        'FAW': 'China',
        'Great Wall': 'China'
    }
    
    df['Country Manufacturer'] = df['Manufacturer'].map(country_manufacturer)
    
    return df

In [80]:
price_predict = preprocessing(auto_cloud)
price_predict

,Car Type,Region,Manufacturer,Transmission,Country Origin,Package,Year,Engine Size,Mileage,Country Manufacturer
0,A,Riyadh,Mercedes,Manual,Saudi,Full,2020,2.0,5000,Germany
1,C,Riyadh,Mercedes,Automatic,Other,Full,2017,7.0,40000,Germany
2,C,Riyadh,Mercedes,Automatic,Saudi,Full,2019,2.0,28000,Germany
3,C,Makkah,Mercedes,Automatic,Saudi,Full,2016,6.3,50000,Germany
4,C,Riyadh,Mercedes,Automatic,Saudi,Full,2015,2.0,110000,Germany
...,...,...,...,...,...,...,...,...,...,...
1746,Land Cruiser Pickup,Riyadh,Toyota,Manual,Saudi,Semi Full,2012,4.0,300000,Japan
1747,Land Cruiser Pickup,Riyadh,Toyota,Manual,Other,Semi Full,2016,5.0,184,Japan
1748,Land Cruiser Pickup,Qassim,Toyota,Manual,Gulf Arabic,Semi Full,2012,4.0,300,Japan
1749,Land Cruiser Pickup,Tabouk,Toyota,Manual,Gulf Arabic,Semi Full,2015,4.0,66000,Japan


In [81]:
predict_file = loaded_model.predict(price_predict)
predict_file

array([189444.81 , 180914.47 , 152631.14 , ...,  45800.562,  88003.266,
        77758.41 ], dtype=float32)

In [82]:
price_predict['Price Prediction'] = predict_file
price_predict

,Car Type,Region,Manufacturer,Transmission,Country Origin,Package,Year,Engine Size,Mileage,Country Manufacturer,Price Prediction
0,A,Riyadh,Mercedes,Manual,Saudi,Full,2020,2.0,5000,Germany,189444.812500
1,C,Riyadh,Mercedes,Automatic,Other,Full,2017,7.0,40000,Germany,180914.468750
2,C,Riyadh,Mercedes,Automatic,Saudi,Full,2019,2.0,28000,Germany,152631.140625
3,C,Makkah,Mercedes,Automatic,Saudi,Full,2016,6.3,50000,Germany,148002.125000
4,C,Riyadh,Mercedes,Automatic,Saudi,Full,2015,2.0,110000,Germany,83798.554688
...,...,...,...,...,...,...,...,...,...,...,...
1746,Land Cruiser Pickup,Riyadh,Toyota,Manual,Saudi,Semi Full,2012,4.0,300000,Japan,62309.644531
1747,Land Cruiser Pickup,Riyadh,Toyota,Manual,Other,Semi Full,2016,5.0,184,Japan,86584.156250
1748,Land Cruiser Pickup,Qassim,Toyota,Manual,Gulf Arabic,Semi Full,2012,4.0,300,Japan,45800.562500
1749,Land Cruiser Pickup,Tabouk,Toyota,Manual,Gulf Arabic,Semi Full,2015,4.0,66000,Japan,88003.265625


## Sent to BigQuery

In [85]:
table_id = 'saudi_usedcars_price_predicted'
client = bigquery.Client()
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

price_predict.columns = ['_'.join(i.split(' ')) for i in price_predict.columns]

job = client.load_table_from_dataframe(price_predict, table_full_id)

job.result()
print(f'loaded {job.output_rows} rows into {table_full_id}')

loaded 1751 rows into dti-ds.shafa_dataset_037.saudi_usedcars_price_predicted


<div style="text-align: center;">
  <h1 style="background-color: grey; color: white; padding: 10px;"><strong>Thank You! :)</strong></h1>
</div>